In [1]:
from python_backend.utils_jobs import get_jobs

In [2]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from scipy.stats import norm
import geopandas as gpd
import copy
import sys
sys.path.insert(0, 'D:/modules/thesis/resources/gp_pref_elicit/gp_utilities/utils_data')


In [3]:
# original dataset
sidewalk = gpd.read_file('D:/modules/thesis/data/Sidewalk_width_crossings_small.geojson')
sidewalk

,id,0.9-1.8m,1.8-2.9m,<0.9m,>2.9m,crossing,length,obstacle_free_width,unknown,geometry
0,0,0,0,0,1,0,9.99,>2.9m,0,"LINESTRING (120548.61203 486088.19578, 120548...."
1,1,0,0,0,1,0,3.64,>2.9m,0,"LINESTRING (120558.58273 486088.59136, 120558...."
2,2,1,0,0,0,0,4.30,0.9-1.8m,0,"LINESTRING (120554.77791 486105.08163, 120555...."
3,3,1,0,0,0,0,3.20,0.9-1.8m,0,"LINESTRING (120561.12010 486102.03679, 120561...."
4,4,0,0,0,0,0,9.99,unknown,1,"LINESTRING (120549.11715 486040.41439, 120549...."
...,...,...,...,...,...,...,...,...,...,...
1134,1134,0,0,0,1,1,8.41,>2.9m,0,"LINESTRING (120971.41298 485981.36204, 120971...."
1135,1135,0,0,0,1,1,25.81,>2.9m,0,"LINESTRING (120916.43844 486032.00808, 120890...."
1136,1136,0,0,0,1,1,9.47,>2.9m,0,"LINESTRING (120940.10969 486013.85196, 120940...."
1137,1137,0,0,0,1,1,8.41,>2.9m,0,"LINESTRING (120971.64289 485989.76706, 120971...."


In [4]:
# dataset with only columns that are needed 
dataset1 = sidewalk[['0.9-1.8m', '1.8-2.9m', '<0.9m', '>2.9m', 'crossing', 'length', 'obstacle_free_width', 'unknown']].copy()
dataset1

,0.9-1.8m,1.8-2.9m,<0.9m,>2.9m,crossing,length,obstacle_free_width,unknown
0,0,0,0,1,0,9.99,>2.9m,0
1,0,0,0,1,0,3.64,>2.9m,0
2,1,0,0,0,0,4.30,0.9-1.8m,0
3,1,0,0,0,0,3.20,0.9-1.8m,0
4,0,0,0,0,0,9.99,unknown,1
...,...,...,...,...,...,...,...,...
1134,0,0,0,1,1,8.41,>2.9m,0
1135,0,0,0,1,1,25.81,>2.9m,0
1136,0,0,0,1,1,9.47,>2.9m,0
1137,0,0,0,1,1,8.41,>2.9m,0


In [5]:
# multiplying all columns with the value in the length column as we are going towards six objectives
# columns '0.9-1.8m', '1.8-2.9m', '<0.9m', '>2.9m' and 'unknown' are path lengths of respective widths
boolean_columns = ['0.9-1.8m', '1.8-2.9m', '<0.9m', '>2.9m', 'unknown']
dataset1[boolean_columns] = dataset1[boolean_columns].mul(dataset1['length'], axis = 0)
dataset1

,0.9-1.8m,1.8-2.9m,<0.9m,>2.9m,crossing,length,obstacle_free_width,unknown
0,0.0,0.0,0.0,9.99,0,9.99,>2.9m,0.00
1,0.0,0.0,0.0,3.64,0,3.64,>2.9m,0.00
2,4.3,0.0,0.0,0.00,0,4.30,0.9-1.8m,0.00
3,3.2,0.0,0.0,0.00,0,3.20,0.9-1.8m,0.00
4,0.0,0.0,0.0,0.00,0,9.99,unknown,9.99
...,...,...,...,...,...,...,...,...
1134,0.0,0.0,0.0,8.41,1,8.41,>2.9m,0.00
1135,0.0,0.0,0.0,25.81,1,25.81,>2.9m,0.00
1136,0.0,0.0,0.0,9.47,1,9.47,>2.9m,0.00
1137,0.0,0.0,0.0,8.41,1,8.41,>2.9m,0.00


In [6]:
# wherever the crossing column has a boolean 1, all the other widths have a value 0 as widths don't define the crossings
crossing_col = dataset1['crossing'] == 1
dataset1.loc[crossing_col, dataset1.columns != 'crossing'] = 0
dataset1

,0.9-1.8m,1.8-2.9m,<0.9m,>2.9m,crossing,length,obstacle_free_width,unknown
0,0.0,0.0,0.0,9.99,0,9.99,>2.9m,0.00
1,0.0,0.0,0.0,3.64,0,3.64,>2.9m,0.00
2,4.3,0.0,0.0,0.00,0,4.30,0.9-1.8m,0.00
3,3.2,0.0,0.0,0.00,0,3.20,0.9-1.8m,0.00
4,0.0,0.0,0.0,0.00,0,9.99,unknown,9.99
...,...,...,...,...,...,...,...,...
1134,0.0,0.0,0.0,0.00,1,0.00,0,0.00
1135,0.0,0.0,0.0,0.00,1,0.00,0,0.00
1136,0.0,0.0,0.0,0.00,1,0.00,0,0.00
1137,0.0,0.0,0.0,0.00,1,0.00,0,0.00


In [7]:
# dropping the length column as we have its values multiplied with other columns, so we have 6 objectives instead of 7
new_dataset = dataset1.drop('length', axis=1)
new_dataset

,0.9-1.8m,1.8-2.9m,<0.9m,>2.9m,crossing,obstacle_free_width,unknown
0,0.0,0.0,0.0,9.99,0,>2.9m,0.00
1,0.0,0.0,0.0,3.64,0,>2.9m,0.00
2,4.3,0.0,0.0,0.00,0,0.9-1.8m,0.00
3,3.2,0.0,0.0,0.00,0,0.9-1.8m,0.00
4,0.0,0.0,0.0,0.00,0,unknown,9.99
...,...,...,...,...,...,...,...
1134,0.0,0.0,0.0,0.00,1,0,0.00
1135,0.0,0.0,0.0,0.00,1,0,0.00
1136,0.0,0.0,0.0,0.00,1,0,0.00
1137,0.0,0.0,0.0,0.00,1,0,0.00


In [8]:
new_dataset['obstacle_free_width'].unique()


array(['>2.9m', '0.9-1.8m', 'unknown', '1.8-2.9m', '<0.9m', 0],
      dtype=object)

In [9]:
new_dataset['obstacle_free_width'].value_counts()

>2.9m       324
1.8-2.9m    277
unknown     180
0           174
0.9-1.8m    141
<0.9m        43
Name: obstacle_free_width, dtype: int64

In [10]:
categorize_obstacles = {"obstacle_free_width": {'0':0, '0.9-1.8m':1, '1.8-2.9m':2, '<0.9m':3,'>2.9m':4,'unknown':5}}
new_dataset = new_dataset.replace(categorize_obstacles)
new_dataset

,0.9-1.8m,1.8-2.9m,<0.9m,>2.9m,crossing,obstacle_free_width,unknown
0,0.0,0.0,0.0,9.99,0,4,0.00
1,0.0,0.0,0.0,3.64,0,4,0.00
2,4.3,0.0,0.0,0.00,0,1,0.00
3,3.2,0.0,0.0,0.00,0,1,0.00
4,0.0,0.0,0.0,0.00,0,5,9.99
...,...,...,...,...,...,...,...
1134,0.0,0.0,0.0,0.00,1,0,0.00
1135,0.0,0.0,0.0,0.00,1,0,0.00
1136,0.0,0.0,0.0,0.00,1,0,0.00
1137,0.0,0.0,0.0,0.00,1,0,0.00


In [11]:
"""
    Gaussian process with a probit likelihood for pairwise comparisons.
        :param num_objectives:      number of objectives of input for utility function we want to approximate
        :param std_noise:           std of the normal distributed noise we assume for the utility function
        :param kernel_width:        parameter for kernel width, default is 0.15
        :param prior_mean_type:     prior mean function type (zero/linear), default is zero
        :param seed:                seed for random state
        """

num_objectives=7 
std_noise=0.01
kernel_width=0.15
prior_mean_type='zero' 
seed=None
random_state = RandomState(seed)

# variables for the observed data
datapoints = None
comparisons = None

# approximate utility values of the datapoints
utility_vals = None

# covariance matrix of datapoints
cov_mat = None
cov_mat_inv = None

# hessian (second derivative) of the pairwise likelihoods for observed data
hess_likelihood = None
hess_likelihood_inv = None

# needed for predictive distribution (cov - hess_likelihood_inv)^(-1)
pred_cov_factor = None

In [12]:
def format_data(data, num_objectives):
    """
    Bring data into right format to be consistent
    :param data:
    :param num_objectives:
    :return data:           reformatted data matrix
    """
    data = np.array(data)
    if data.ndim == 0:
        data = np.zeros((1, num_objectives)) + data
    # if we get only one datapoint, make columns the objectives
    elif data.ndim == 1:
        if len(data) == num_objectives:
            data = data[np.newaxis, :]
        else:
            data = data[:, np.newaxis]
    # make sure the columns are the objectives
    elif data.shape[1] != num_objectives and data.shape[0] == num_objectives:
        data = data.T
    elif data.shape[1] == num_objectives:
        data = data
    else:
        raise RuntimeError('Data does not seem to have the right number of objectives.')

    return data

In [13]:
formatted_data = format_data(new_dataset, 7)
formatted_data

array([[0. , 0. , 0. , ..., 0. , 4. , 0. ],
       [0. , 0. , 0. , ..., 0. , 4. , 0. ],
       [4.3, 0. , 0. , ..., 0. , 1. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 1. , 0. , 0. ],
       [0. , 0. , 0. , ..., 1. , 0. , 0. ],
       [0. , 0. , 0. , ..., 1. , 0. , 0. ]])

In [14]:
def prior_mean(x):
        """
        Prior mean function
        :param x:   input of size [num_datapoints x num_objectives]
        :return:
        """
        
        x = format_data(x, num_objectives)
        m = np.zeros(x.shape[0])
        if prior_mean_type == 'linear':
            m += np.sum(x, axis=1) / num_objectives
        else:
            TypeError('Prior mean type not understood.')
        return m

In [15]:
prior_mean(7973)

array([0.])

In [16]:
def _kernel(x1, x2):
        """
        Squared exponential kernel function
        :param x1:
        :param x2:
        :return:
        """
   
        x1 = format_data(x1, num_objectives)
        x2 = format_data(x2, num_objectives)
        k = 0.8**2 * np.exp(-(1. / (2. * (kernel_width ** 2))) * np.linalg.norm(x1 - x2, axis=1) ** 2)
        return k

In [17]:
# _kernel()

In [18]:
def _evaluate_prior(input_points):
        """
        Given some datapoints, evaluate the prior
        :param input_points:    input datapoints at which to evaluate prior
        :return:                predictive mean and covariance at the given inputs
        """
        pred_mean = prior_mean(input_points)
        num_inputs = input_points.shape[0]
        pred_cov = _kernel(np.repeat(input_points, num_inputs, axis=0),
                                np.tile(input_points, (num_inputs, 1))).reshape((num_inputs, num_inputs))
        return pred_mean, pred_cov

In [19]:
# _evaluate_prior()

In [20]:
def _cov_mat(x1, x2=None, noise=True):
        """
        Covariance matrix for preference data using the kernel function.
        :param x1:      datapoints for which to compute covariance matrix
        :param x2:      if None, covariance matrix will be square for the input x1
                        if not None, covariance will be between x1 (rows) and x2 (cols_
        :param noise:   whether to add noise to the diagonal of the covariance matrix
        :return:
        """
       
        if x2 is None:
            x2 = x1
        else:  # if x1 != x2 we don't add noise!
            noise = False

        x1 = format_data(x1, num_objectives)
        x2 = format_data(x2, num_objectives)

        cov_mat = _kernel(np.repeat(x1, x2.shape[0], axis=0), np.tile(x2, (x1.shape[0], 1)))
        cov_mat = cov_mat.reshape((x1.shape[0], x2.shape[0]))

        if noise:
            cov_mat += std_noise ** 2 * np.eye(cov_mat.shape[0])

        return cov_mat

In [21]:
def _compute_hess_likelihood_entry(m, n, z, z_logpdf, z_logcdf):
        """
        Get a single entry for the Hessian matrix at indices (m,n)
        :param m:
        :param n:
        :param f:
        :return:
        """
        
        h_x_m = np.array(comparisons[:, 0] == m, dtype=int) - np.array(comparisons[:, 1] == m, dtype=int)
        h_x_n = np.array(comparisons[:, 0] == n, dtype=int) - np.array(comparisons[:, 1] == n, dtype=int)
        p = h_x_m * h_x_n * (np.exp(2.*z_logpdf - 2.*z_logcdf) + z * np.exp(z_logpdf - z_logcdf))
        c = - np.sum(p) / (2 * std_noise**2)
        return c

In [22]:
def _compute_hess_likelihood(z=None):
        """
        Compute the hessian of the likelihood
        :return:
        """
      
        if z is None:
            # compute z
            f_winner = np.array([utility_vals[comparisons[i, 0]] for i in range(comparisons.shape[0])])
            f_loser = np.array([utility_vals[comparisons[i, 1]] for i in range(comparisons.shape[0])])
            z = (f_winner - f_loser) / (np.sqrt(2) * std_noise)

        z_logpdf = norm.logpdf(z)
        z_logcdf = norm.logcdf(z)

        # initialise with zeros
        lambda_mat = np.zeros((datapoints.shape[0], datapoints.shape[0]))

        # build up diagonal for pairs (xi, xi)
        diag_arr = np.array([_compute_hess_likelihood_entry(m, m, z, z_logpdf, z_logcdf) for m in
                             range(datapoints.shape[0])])
        np.fill_diagonal(lambda_mat, diag_arr)  # happens in-place

        # go through the list of comparisons collected so far and update lambda
        for k in range(comparisons.shape[0]):
            m = comparisons[k, 0]  # winner
            n = comparisons[k, 1]  # loser
            lambda_mat[m, n] = _compute_hess_likelihood_entry(m, n, z, z_logpdf, z_logcdf)
            lambda_mat[n, m] = _compute_hess_likelihood_entry(n, m, z, z_logpdf, z_logcdf)

        # add jitter term to make lambda positive definite for computational stability
        lambda_mat += np.eye(datapoints.shape[0]) * 0.01

        return lambda_mat

In [23]:
def _compute_posterior(self):
        """
        Approximate the posterior distribution
        :return:    MAP of the gp values at current datapoints
        """

        converged = False
        try_no = 0

        f_map = None

        # using Newton-Raphson, approximate f_MAP
        while not converged and try_no < 1:

            # randomly initialise f_map
            f_map = self.random_state.uniform(0., 1., self.datapoints.shape[0])

            for m in range(100):

                # compute z
                f_winner = np.array([f_map[self.comparisons[i, 0]] for i in range(self.comparisons.shape[0])])
                f_loser = np.array([f_map[self.comparisons[i, 1]] for i in range(self.comparisons.shape[0])])
                z = (f_winner - f_loser) / (np.sqrt(2) * self.std_noise)
                z_logpdf = norm.logpdf(z)
                z_logcdf = norm.logcdf(z)

                # compute b
                h_j = np.array([np.array(self.comparisons[:, 0] == j, dtype=int) -
                                np.array(self.comparisons[:, 1] == j, dtype=int) for j in
                                range(self.datapoints.shape[0])])
                b = np.sum(h_j * np.exp(z_logpdf - z_logcdf), axis=1) / (np.sqrt(2) * self.std_noise)

                # compute gradient g
                g = - np.dot(self.cov_mat_inv, (f_map - self.prior_mean(self.datapoints))) + b

                # compute approximation of the hessian of the posterior
                hess_likelihood = self._compute_hess_likelihood(z)
                hess_posterior = - self.cov_mat_inv + hess_likelihood
                try:
                    hess_posterior_inv = np.linalg.inv(hess_posterior)
                except:
                    hess_posterior_inv = np.linalg.pinv(hess_posterior)

                # perform update
                update = np.dot(hess_posterior_inv, g)
                f_map -= update

                # stop criterion
                if np.linalg.norm(update) < 0.0001:
                    converged = True
                    break

            if not converged:
                print("Did not converge.")
                try_no += 1

        return f_map


In [24]:
def update(dataset):
        """
        Update the Gaussian process using the given data;
        call this before calling sample() or get_predictive_params()
        :param dataset:
        """
        datapoints = dataset.datapoints
        comparisons = dataset.comparisons

        # compute the covariance matrix given the new datapoints
        cov_mat = _cov_mat(datapoints)
        cov_mat_inv = np.linalg.inv(cov_mat)

        # compute the map estimate of f
        utility_vals = _compute_posterior()

        # compute the hessian of the likelihood given f_MAP
        hess_likelihood = _compute_hess_likelihood()
        try:
            hess_likelihood_inv = np.linalg.inv(hess_likelihood)
        except:
            hess_likelihood_inv = np.linalg.pinv(hess_likelihood)

        pred_cov_factor = np.linalg.inv(cov_mat - hess_likelihood_inv)


In [25]:
def get_predictive_params(x_new, pointwise):
        """
        Returns the predictive parameters (mean, variance) of the Gaussian distribution
        at the given datapoints;
        :param x_new:       the points for which we want the predictive params
        :param pointwise:   whether we want pointwise variance or the entire covariance matrix
        :return:
        """
        # bring input points into right shape
        x_new = format_data(x_new, num_objectives=7)

        datapoints = None
        utility_vals = None
        cov_mat_inv = None
        
        # if we don't have any data yet, use prior GP to make predictions
        if datapoints is None or utility_vals is None:
            pred_mean, pred_var = _evaluate_prior(x_new)

        # otherwise compute predictive mean and covariance
        else:
            cov_xnew_x = _cov_mat(x_new, datapoints, noise=False)
            cov_x_xnew = _cov_mat(datapoints, x_new, noise=False)
            cov_xnew = _cov_mat(x_new, noise=False)
            pred_mean = prior_mean(x_new) + np.dot(np.dot(cov_xnew_x, cov_mat_inv),
                                                        (utility_vals - prior_mean(datapoints)))
            pred_var = cov_xnew - np.dot(np.dot(cov_xnew_x, pred_cov_factor), cov_x_xnew)

        if pointwise:
            pred_var = pred_var.diagonal()

        return pred_mean, pred_var

In [26]:
def array_in_matrix(arrays, matrix, rounding_accuracy=5):
    """
    Checks if the array a is in the (rows of) the matrix m
    :param arrays:
    :param matrix:
    :param rounding_accuracy:   we will round the values of the array
                                and the matrix until this many digits after
                                the comma; default is 10
    :return:
    """
    if len(matrix) == 0:
        return False
    arrays = np.array(arrays)
    if matrix.ndim == 1:
        matrix = matrix[np.newaxis, :]
    if arrays.ndim == 1:
        if matrix.shape[0] == 0:
            return 0
        array = np.round(arrays, rounding_accuracy)
        matrix = np.round(matrix, rounding_accuracy)
        return int(np.sum(np.sum(np.abs(matrix - array), axis=1) == 0))
    else:
        if matrix.shape[0] == 0:
            return np.zeros(arrays.shape[0])
        arrays = np.round(arrays, rounding_accuracy)
        matrix = np.round(matrix, rounding_accuracy)
        diff = np.abs(matrix[:, np.newaxis] - arrays)
        diff = np.sum(diff, axis=2)
        a_in_m = np.sum(diff == 0, axis=0) > 0
        return np.array(a_in_m, dtype=int)


In [27]:
def scale_to_unit_interval(x, y=None):
    """
    Scale the values in x to lie between 0 and 1;
    use min and max values from x if y is none;
    else use min and max values from y
    :param x:           values to transform to lie between 0 and 1
    :param y:           optional; if not None we use min(y) and max(y)
                        to rescale_on_ccs values in x
    :return x_scaled:   the values of x, scaled to the unit interval
    """
    min_val = np.min(x) if y is None else np.min(y)
    max_val = np.max(x) if y is None else np.max(y)

    if min_val != max_val:
        x_scaled = (x - min_val) / (max_val - min_val)
    # if all values in x have the same value and this is between 0 and 1, return x
    elif 1 > max_val > 0:
        x_scaled = x
    # if all values in x have the same value which is not between 0 and 1, return zeros
    else:
        x_scaled = np.zeros(x.shape)

    return x_scaled

In [28]:
def sample(sample_points):
        """
        Get a sample from the current GP at the given points.
        :param sample_points:   the points at which we want to take the sample
        :return:                the values of the GP sample at the input points
        """
        # bring sample points in right shape
        sample_points = format_data(sample_points, num_objectives=7)

        # get the mean and the variance of the predictive (multivariate gaussian) distribution at the sample points
        mean, var = get_predictive_params(sample_points, pointwise=False)

        # sample from the multivariate gaussian with the given parameters
        f_sample = random_state.multivariate_normal(mean, var, 1)[0]

        return f_sample

In [29]:
format_data(data=new_dataset, num_objectives=7)

array([[0. , 0. , 0. , ..., 0. , 4. , 0. ],
       [0. , 0. , 0. , ..., 0. , 4. , 0. ],
       [4.3, 0. , 0. , ..., 0. , 1. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 1. , 0. , 0. ],
       [0. , 0. , 0. , ..., 1. , 0. , 0. ],
       [0. , 0. , 0. , ..., 1. , 0. , 0. ]])

In [30]:
sampled_points = sample(sample_points=new_dataset)
sampled_points

array([ 1.58431576,  0.33976179, -0.35958531, ..., -0.90066849,
       -0.90066864, -0.90066842])

In [31]:
"""
        An acquirer for a discrete set of points, using the expected improvement.
        :param input_domain:     (np.array) the datapoints on which the discrete acquirer is defined.
        :param query_type:       (str) the query type of the current experiment (pairwise/ranking/clustering/top_rank)
        :param seed:             (int) random seed
        :param acquisition_type: (str) type of acquisition function, can be "expected improvement" or "thompson sampling"
        """
acquisition_type = 'expected improvement'
from webInterface.python_backend import utils_jobs

input_domain = utils_jobs.get_jobs()
input_domain = copy.deepcopy(input_domain)
query_type = 'pairwise'
acq_type = acquisition_type
random_state = np.random.RandomState(seed)
history = np.empty((0, input_domain.shape[1]))

In [32]:
def get_expected_improvement(datapoints, datapoints_hist, xi=0.01):
    """
    Calculate the expected improvement of datapoints given a GP
    :param datapoints:          the datapoints for which we want to get the EI
    :param gaussian_process:    the gaussian process model
    :param datapoints_hist:     datapoints we have already queried
    :param xi:                  hyperparameter
    :return:    expected improvement for given datapoints
    """

    # initialise the expected improvement vector
    exp_impr = np.zeros(datapoints.shape[0])

    # predicted mean and variance at datapoints
    pred_mean, pred_var = get_predictive_params(datapoints, pointwise=True)

    # get the value of the point (from our history) with the highest predicted mean
    max_f = 0
    if datapoints_hist.shape[0] > 0:
        max_f = np.max(get_predictive_params(datapoints_hist, pointwise=True)[0])

    mfxi = pred_mean[pred_var != 0] - max_f - xi
    z = mfxi / pred_var[pred_var != 0]
    cdf_z = norm.cdf(z)
    pdf_z = norm.pdf(z)

    # if the variance is zero, the EI is defined as zero
    exp_impr[pred_var != 0] = mfxi * cdf_z + pred_var[pred_var != 0] * pdf_z

    return exp_impr

In [33]:
def get_start_points(gaussian_process):
        """
        Get datapoints which should be queried first
        :param gaussian_process:
        :return:
        """
        # get the expected improvement of the whole input domain (in batches, might be slow otherwise)
        exp_impr = np.zeros(input_domain.shape[0])
        history = np.empty((0, input_domain.shape[1]))
        batch_size = 64
        for curr_idx in range(0, input_domain.shape[0]+batch_size, batch_size):
            exp_impr[curr_idx:curr_idx+batch_size] = get_expected_improvement(input_domain[curr_idx:curr_idx+batch_size], gaussian_process, history)

        # get the best expected improvement things
        best_points = input_domain[exp_impr == np.max(exp_impr)]
        if len(best_points) > 1:
            indices = random_state.choice(range(len(best_points)), 2, replace=False)
            datapoint1 = best_points[indices[0]]
            datapoint2 = best_points[indices[1]]
        else:
            datapoint1 = best_points[0]
            second_best_points = input_domain[exp_impr == np.sort(exp_impr)[-2]]
            idx = random_state.choice(range(len(second_best_points)), replace=False)
            datapoint2 = second_best_points[idx]

        history = np.vstack((history, datapoint1, datapoint2))
        return datapoint1, datapoint2

In [34]:
def exclude_points_pairwise(dataset):
    """
    When we do pairwise comparisons, we exclude all jobs the current max
    was already compared to
    :param dataset:
    :return:
    """
    job_max = dataset.datapoints[dataset.comparisons[-1, 0]]
    exclude = job_max[np.newaxis, :]
    job_max_idx = dataset.get_index(job_max)
    for comp in dataset.comparisons:
        if job_max_idx in list(comp):
            new_idx = comp[1 - list(comp).index(job_max_idx)]
            exclude = np.vstack((exclude, dataset.datapoints[new_idx]))
    return exclude


In [35]:
def exclude_points_ranking(dataset):
    """
    When we do ranking, none of the existing jobs should be added again
    :param dataset:
    :return:
    """
    return dataset.datapoints

In [36]:
def exclude_points(query_type, dataset):
    if query_type == 'pairwise':
        return exclude_points_pairwise(dataset)
    elif query_type == 'ranking' or query_type == 'clustering' or query_type == 'top_rank' or query_type == 'best_worst':
        return exclude_points_ranking(dataset)
    else:
        raise NotImplementedError("Query type {} unknown.".format(query_type))

In [37]:
def get_next_point(self, gaussian_process, dataset):
        """
        Get the next datapoint to query
        :param gaussian_process:
        :param dataset:
        :return:
        """
        # points which can't be queried next
        exclude = exclude_points(self.query_type, dataset)

        if self.acq_type == 'expected improvement':
            next_point = self.get_next_point_EI(gaussian_process, exclude)
        elif self.acq_type == 'thompson sampling':
            next_point = self.get_next_point_thompson(gaussian_process, exclude)
        else:
            raise RuntimeError()

        # append to history
        self.history = np.vstack((self.history, next_point))

        return next_point

In [38]:
def get_next_point_EI(self, gaussian_process, exclude):
        """
        Returns the next datapoint to query, using expected improvement
        :param gaussian_process:
        :param exclude:
        :return:
        """
        # get the expected improvement of the whole input domain (in batches, might be slow otherwise)
        expected_improvement = np.zeros(self.input_domain.shape[0])
        batch_size = 64
        for curr_idx in range(0, self.input_domain.shape[0]+batch_size, batch_size):
            expected_improvement[curr_idx:curr_idx+batch_size] = get_expected_improvement(self.input_domain[curr_idx:curr_idx+batch_size], gaussian_process, self.history)

        # find the point with the highest EI, and which can be queried next
        next_point = self.input_domain[np.argmax(expected_improvement)]
        next_point_idx = 1
        while array_in_matrix(next_point, exclude):
            if next_point_idx >= self.input_domain.shape[0]:
                "Run out of points to display next. You should end the experiment."
                break
            next_point = self.input_domain[np.argsort(-expected_improvement)[next_point_idx]]
            next_point_idx += 1

        return next_point

In [39]:
# get_start_points(sampled_points)

In [40]:
import gp_utilities.utils_ccs as gp_utils 

In [41]:
def get_pcs_grid(ccs_size, num_objectives, eucledian_dist=0.05, min_size=100, seed=None):

    pcs_grid = np.empty((0, num_objectives))
    random_state = np.random.RandomState(seed)
    while pcs_grid.shape[0] < min_size:

        # get a CCS
        ccs = get_ccs(num_objectives, ccs_size, seed)

        # get a pcs grid from that ccs
        pcs_grid = get_pcs_grid_from_ccs(ccs, eucledian_dist, seed)

        seed += random_state.randint(0, 10000, 1)[0]

        if pcs_grid is None:
            pcs_grid = np.empty((0, num_objectives))

    return pcs_grid


In [42]:
# gp_utils.get_ccs(2, 20)

In [72]:
synthetic_pcs = np.array([[0.14370116, 0.99159928],
       [0.9797389 , 0.2242916 ],
       [0.        , 1.        ],
       [0.91055917, 0.45020785],
       [0.59678925, 0.81854996],
       [1.        , 0.        ],
       [0.94198057, 0.352479  ],
       [0.81501748, 0.65358114],
       [0.99814566, 0.05429028],
       [0.33305315, 0.94955291],
       [0.28860669, 0.96123215],
       [0.99999796, 0.02591092],
       [0.98910769, 0.14092867],
       [0.18584726, 0.98334638],
       [0.05210043, 0.99838732],
       [0.87761802, 0.52114756],
       [0.74002719, 0.7144284 ],
       [0.21487083, 0.97724899],
       [0.43622937, 0.90230767],
       [0.99525346, 0.08213535]])
synthetic_pcs 

array([[0.14370116, 0.99159928],
       [0.9797389 , 0.2242916 ],
       [0.        , 1.        ],
       [0.91055917, 0.45020785],
       [0.59678925, 0.81854996],
       [1.        , 0.        ],
       [0.94198057, 0.352479  ],
       [0.81501748, 0.65358114],
       [0.99814566, 0.05429028],
       [0.33305315, 0.94955291],
       [0.28860669, 0.96123215],
       [0.99999796, 0.02591092],
       [0.98910769, 0.14092867],
       [0.18584726, 0.98334638],
       [0.05210043, 0.99838732],
       [0.87761802, 0.52114756],
       [0.74002719, 0.7144284 ],
       [0.21487083, 0.97724899],
       [0.43622937, 0.90230767],
       [0.99525346, 0.08213535]])

In [73]:
import gaussian_process as gp

In [74]:
gp_for_2 = gp.GPPairwise(2) 
gp_for_2 

In [75]:
gp_for_2 = gp_for_2.sample(synthetic_pcs)
gp_for_2
# output is sample utility value according to zero mean

array([-0.46944601,  0.03079761, -0.47246461, -0.74973023,  0.96769496,
        0.15971465,  0.08412812, -1.44560274, -0.10811851, -0.32277358,
       -0.39974255,  0.01161056, -0.17467415, -0.4866124 , -0.43274567,
       -1.353991  , -0.4412894 , -0.47911537, -0.04007822, -0.17797565])

In [76]:
import dataset as data 

In [77]:
data1 = data.DatasetPairwise(2)
data1

In [78]:
data1 = data1.add_single_comparison(1.6973058, -0.30532143)

In [52]:
gp_for_2.sample(synthetic_pcs)

array([-0.44648558,  0.88483424, -0.7920268 , -0.85117523,  1.42548193,
       -0.20856296,  0.06016535,  0.0065365 ,  0.13707259,  0.37480051,
        0.34216918, -0.05853126,  0.72095593, -0.12585086, -0.85874065,
       -0.82310914,  0.4174746 ,  0.06877336,  0.6318598 ,  0.34170693])

In [59]:
data2 = data.DatasetPairwise(2)
data2 = data2.add_single_comparison(1.42548193, -0.85874065)

In [62]:
ranking = data.DatasetPairwise(2)
ranking = ranking.add_ranked_preferences(synthetic_pcs)
ranking